In [52]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, OneHotEncoder, StringIndexer
import pandas as pd
import numpy as np
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.types import ArrayType, DoubleType
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression

### Read the pickup data

In [ ]:
postgresql_jdbc_jar = r"C:/Program Files/PostgreSQL/17/postgresql-42.7.4.jar"
spark = SparkSession.builder.appName('FeatureEngineering')\
                            .config("spark.jars", postgresql_jdbc_jar) \
                            .config("spark.driver.extraClassPath", postgresql_jdbc_jar) \
                            .config("spark.driver.memory", "8g")\
                            .config("spark.executor.memory", "8g")\
                            .config("spark.executor.cores", "4")\
                            .getOrCreate()

In [54]:
# Database connection parameters
url = "jdbc:postgresql://localhost:5432/postgres"
properties = {
    "user": "postgres",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

# Query to filter cities Hangzhou and Shanghai
query = "(SELECT * FROM pickup_data WHERE city IN ('Chongqing', 'Shanghai', 'Yantai', 'Jilin')) AS filtered_data"

# Load the data into a PySpark DataFrame
df_pickup = spark.read.jdbc(url=url, table=query, properties=properties)

# Show the first few rows
row_count = df_pickup.count()
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 4005691


In [57]:
df_pickup.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- courier_id: integer (nullable = true)
 |-- accept_time: timestamp (nullable = true)
 |-- time_window_start: timestamp (nullable = true)
 |-- time_window_end: timestamp (nullable = true)
 |-- aoi_id: integer (nullable = true)
 |-- aoi_type: integer (nullable = true)
 |-- pickup_time: timestamp (nullable = true)
 |-- ds: integer (nullable = true)
 |-- lng: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- pickup_gps_lng: double (nullable = true)
 |-- pickup_gps_lat: double (nullable = true)
 |-- accept_gps_lng: double (nullable = true)
 |-- accept_gps_lat: double (nullable = true)
 |-- pickup_eta_minutes: double (nullable = true)



In [58]:
df_pickup = df_pickup.drop('pickup_eta_minutes')

### Time-Based Features

In [60]:
from pyspark.sql.functions import col, mean

# Extracting hour, day of the week, and month from the `accept_time` column
df_pickup = df_pickup.withColumn("hour_of_day", F.hour(df_pickup["accept_time"])) \
       .withColumn("day_of_week", F.dayofweek(df_pickup["accept_time"])) \
       .withColumn("month", F.month(df_pickup["accept_time"]))

# Calculate time taken for pickup (in minutes) which is already calculated in pickup_eta_minutes

# Calculate the time difference between expected and actual pickup time in minutes
# expected_pickup_time = time_window_start
# actual pickup time = pickup_time
# A positive value means the pickup happened after the expected time (late pickup), 
# while a negative value means it happened earlier than expected (early pickup).

# # Compute avg_pickup_time_minutes
# avg_pickup_time = df_pickup.select(expr("percentile_approx(pickup_eta_minutes, 0.50)")).collect()[0][0]

# # Compute pickup_time_delay
# df_pickup = df_pickup.withColumn("pickup_time_delay", col("pickup_eta_minutes") - avg_pickup_time)

df_pickup = df_pickup.withColumn("pickup_eta_minutes",
                  (df_pickup["time_window_start"].cast("long") - df_pickup["accept_time"].cast("long")) / 60)

df_pickup = df_pickup.withColumn("pickup_time_delay",
                  (df_pickup["pickup_time"].cast("long") - df_pickup["accept_time"].cast("long")) / 60)

# Show result
df_pickup.select("pickup_eta_minutes", "pickup_time_delay").show(10)

+------------------+-----------------+
|pickup_eta_minutes|pickup_time_delay|
+------------------+-----------------+
|              95.0|             91.0|
|              95.0|             50.0|
|              95.0|            185.0|
|              95.0|             38.0|
|              95.0|             84.0|
|               0.0|             48.0|
|              96.0|             33.0|
|              96.0|             33.0|
|              96.0|             10.0|
|               0.0|             58.0|
+------------------+-----------------+
only showing top 10 rows



In [66]:
from pyspark.sql.functions import col

# Replace 'column_name' with your actual column name
negative_count = df_pickup.filter(col("pickup_eta_minutes") < 0).count()

print(f"Number of negative values in column 'pickup_time_delay': {negative_count}")

Number of negative values in column 'pickup_time_delay': 0


In [65]:
df_pickup = df_pickup.filter(col("pickup_eta_minutes") >= 0)

In [67]:
df_pickup.select("pickup_time_delay").summary().show()


+-------+-----------------+
|summary|pickup_time_delay|
+-------+-----------------+
|  count|          3976707|
|   mean|216.8401511602439|
| stddev|412.8116907220085|
|    min|              0.0|
|    25%|             63.0|
|    50%|            116.0|
|    75%|            192.0|
|    max|          54859.0|
+-------+-----------------+



In [ ]:
# # looking above pickup_time_delay distribution is still highly skewed, with a mean of ~101 minutes but a median (50%) of 0. 
# # The max value of 54,744 minutes (~38 days) suggests there are extreme outliers.
# # Handle Outliers (Max Value = 54,744), Use the 99th percentile to filter extreme delays
# threshold = df_pickup.approxQuantile("pickup_time_delay", [0.99], 0.01)[0]
# df_pickup = df_pickup.filter(col("pickup_time_delay") <= threshold)

In [ ]:
# df_pickup = df_pickup.withColumn(
#     "pickup_time_delay", 
#     F.when(col("pickup_time_delay") < -30, 0).otherwise(col("pickup_time_delay"))
# )

In [68]:
total_rows = df_pickup.count()
print(f"Total number of rows: {total_rows}")

Total number of rows: 3976707


In [26]:
df_pickup.select("hour_of_day", "day_of_week", "month", "pickup_time_delay").show(5)

+-----------+-----------+-----+-----------------+
|hour_of_day|day_of_week|month|pickup_time_delay|
+-----------+-----------+-----+-----------------+
|          7|          7|    6|            529.0|
|          7|          7|    6|             32.0|
|          7|          7|    6|            513.0|
|          7|          7|    6|            115.0|
|          7|          7|    6|            277.0|
+-----------+-----------+-----+-----------------+
only showing top 5 rows



In [69]:
# Filter rows where pickup_time_difference is negative
df_negative_pickups = df_pickup.filter(df_pickup["pickup_time_delay"] < 0)
df_negative_pickups.select("order_id","city", "pickup_time_delay").show(5)

+--------+----+-----------------+
|order_id|city|pickup_time_delay|
+--------+----+-----------------+
+--------+----+-----------------+



### Geospatial Features

#### Calculate the distance between pickup and delivery locations using Haversine Formula.

In [70]:
# Define the Haversine function to calculate distance between two points on the Earth
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    
    dlat = radians(lat2 - lat1)  # Difference in latitudes (radians)
    dlon = radians(lon2 - lon1)  # Difference in longitudes (radians)
    
    # Apply Haversine formula to calculate the "a" value
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    # calculate the central angle between two points
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Compute the distance in KM using the radius of Earth (R) and the central angle (c)
    distance = R * c 
    return distance

# Apply the Haversine function to the DataFrame to calculate the distance in KM
df_pickup = df_pickup.withColumn("pickup_distance_km", 
                                 haversine(df_pickup["pickup_gps_lat"], df_pickup["pickup_gps_lng"],
                                           df_pickup["accept_gps_lat"], df_pickup["accept_gps_lng"]))
# Round the 'pickup_distance_km' column to 2 decimal places
df_pickup = df_pickup.withColumn("pickup_distance_km", round(df_pickup["pickup_distance_km"], 2))

In [71]:
df_pickup.select("order_id","pickup_gps_lat","pickup_gps_lng","accept_gps_lat","accept_gps_lng","pickup_distance_km").show(10)

+--------+--------------+--------------+--------------+--------------+------------------+
|order_id|pickup_gps_lat|pickup_gps_lng|accept_gps_lat|accept_gps_lng|pickup_distance_km|
+--------+--------------+--------------+--------------+--------------+------------------+
| 3764982|      32.57753|     118.38133|      32.70376|     118.54136|             20.53|
| 5767762|      37.56164|     121.27105|      37.54782|     121.25854|              1.89|
| 3477155|      31.01129|     121.21426|      31.00848|     121.19963|              1.43|
| 2909859|       31.3071|      121.4905|      31.30869|     121.49028|              0.18|
| 3520728|      31.23623|     121.72127|      31.23602|     121.71976|              0.15|
|  940606|      32.57753|     118.38133|      32.70376|     118.54136|             20.53|
| 2373275|      29.70774|     107.37643|      29.70717|     107.38724|              1.05|
| 1022562|       31.3081|     121.49443|       31.3077|     121.49027|               0.4|
| 5449301|

#### Cluster regions using K-Means or DBSCAN for better route mapping.

In [72]:
# observing above regions and number of orders they have we can see here is a large variation in the number of rows (packages) across regions, 
# DBSCAN would be a more appropriate clustering algorithm for this scenario as It can handle regions where some areas are densely packed with 
# packages (e.g., urban centers) while others have fewer packages (e.g., rural areas).
# Also it doesn't require the number of clusters to be predefined and can naturally identify clusters of different shapes and sizes.

In [73]:
# Step 1: Select the relevant geographic columns (Longitude and Latitude)
feature_columns = ["pickup_gps_lng", "pickup_gps_lat"]

# Step 2: Vectorize the geographic features into a single vector column 'features'
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_vectorized = vector_assembler.transform(df_pickup)

# Step 4: Apply KMeans with a specified number of clusters (let's say 5)
kmeans = KMeans(k=25, seed=1, featuresCol="features", predictionCol="cluster")
model = kmeans.fit(df_vectorized)

# Step 5: Make predictions (assign clusters to the data)
predictions = model.transform(df_vectorized)

In [74]:
# Step 7: Show some of the prediction results (assigned cluster labels)
predictions.select("order_id", "city", "cluster").show()

+--------+---------+-------+
|order_id|     city|cluster|
+--------+---------+-------+
| 3764982| Shanghai|      2|
| 5767762|   Yantai|      1|
| 3477155| Shanghai|     19|
| 2909859| Shanghai|     20|
| 3520728| Shanghai|      4|
|  940606|Chongqing|      2|
| 2373275|Chongqing|     14|
| 1022562| Shanghai|     20|
| 5449301|Chongqing|     23|
| 1676134|Chongqing|      2|
| 1973550|Chongqing|      0|
| 5278857|    Jilin|     21|
| 2853126|   Yantai|     16|
| 5664753| Shanghai|     20|
|  207390|    Jilin|      2|
| 6118921| Shanghai|      4|
| 3068924|Chongqing|     14|
|  723232|Chongqing|      0|
| 1345159|Chongqing|     23|
| 4397326|Chongqing|      2|
+--------+---------+-------+
only showing top 20 rows



In [75]:
predictions = predictions.drop('features')

### Courier & Package Features

#### Compute the average pickup time per courier

In [76]:
# Group by courier_id and calculate the average pickup time delay (in minutes)
average_pickup_time_per_courier = predictions.groupBy("courier_id").agg(
    F.avg("pickup_time_delay").alias("avg_pickup_time_minutes")
)

# Show the result
predictions.show(5)
predictions = predictions.join(
    average_pickup_time_per_courier, on="courier_id", how="left")

+--------+---------+--------+----------+-------------------+-------------------+-------------------+------+--------+-------------------+---+---------+--------+--------------+--------------+--------------+--------------+-----------+-----------+-----+------------------+-----------------+------------------+-------+
|order_id|region_id|    city|courier_id|        accept_time|  time_window_start|    time_window_end|aoi_id|aoi_type|        pickup_time| ds|      lng|     lat|pickup_gps_lng|pickup_gps_lat|accept_gps_lng|accept_gps_lat|hour_of_day|day_of_week|month|pickup_eta_minutes|pickup_time_delay|pickup_distance_km|cluster|
+--------+---------+--------+----------+-------------------+-------------------+-------------------+------+--------+-------------------+---+---------+--------+--------------+--------------+--------------+--------------+-----------+-----------+-----+------------------+-----------------+------------------+-------+
| 3764982|       21|Shanghai|      5073|2025-08-13 15:25:0

#### Extract package pickup patterns (e.g., peak hours, busiest locations).

#### Peak hours

In [77]:
# Step 1: Group by hour_of_day and count the number of pickups (orders)
pickup_by_hour = predictions.groupBy("hour_of_day").agg(
    F.count("order_id").alias("pickup_order_count")
)

# Step 2: Sort by the pickup count in descending order to find the peak hours
pickup_by_hour_sorted = pickup_by_hour.orderBy(F.col("pickup_order_count").desc())

# Step 3: Join the pickup count by hour to the predictions DataFrame based on hour_of_day
predictions = predictions.join(
    pickup_by_hour_sorted, on="hour_of_day", how="left"
)

# Show the result: predictions with pickup count by hour
predictions.select("order_id", "hour_of_day", "pickup_order_count").show(5)

+--------+-----------+------------------+
|order_id|hour_of_day|pickup_order_count|
+--------+-----------+------------------+
| 3764982|         15|            170936|
| 5767762|         15|            170936|
| 3477155|         15|            170936|
| 2909859|         15|            170936|
| 3520728|         15|            170936|
+--------+-----------+------------------+
only showing top 5 rows



#### Busiest locations

In [78]:
df_busiest_locations = predictions.groupBy("city") \
    .agg(F.count("order_id").alias("city_order_count")) \
    .orderBy(F.desc("city_order_count"))
df_busiest_locations.show(5)

+---------+----------------+
|     city|city_order_count|
+---------+----------------+
| Shanghai|         1409554|
|Chongqing|         1164146|
|   Yantai|         1142467|
|    Jilin|          260540|
+---------+----------------+



In [79]:
# Step 3: Join the pickup count by hour to the predictions DataFrame based on hour_of_day
predictions = predictions.join(
    df_busiest_locations, on="city", how="left"
)

In [80]:
predictions.select("order_id", "city", "city_order_count").show(5)

+--------+---------+----------------+
|order_id|     city|city_order_count|
+--------+---------+----------------+
| 5940108|Chongqing|         1164146|
| 1378407|Chongqing|         1164146|
| 4725304|Chongqing|         1164146|
| 6142927|Chongqing|         1164146|
|  821810|   Yantai|         1142467|
+--------+---------+----------------+
only showing top 5 rows



In [81]:
predictions.printSchema()

root
 |-- city: string (nullable = true)
 |-- hour_of_day: integer (nullable = true)
 |-- courier_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- accept_time: timestamp (nullable = true)
 |-- time_window_start: timestamp (nullable = true)
 |-- time_window_end: timestamp (nullable = true)
 |-- aoi_id: integer (nullable = true)
 |-- aoi_type: integer (nullable = true)
 |-- pickup_time: timestamp (nullable = true)
 |-- ds: integer (nullable = true)
 |-- lng: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- pickup_gps_lng: double (nullable = true)
 |-- pickup_gps_lat: double (nullable = true)
 |-- accept_gps_lng: double (nullable = true)
 |-- accept_gps_lat: double (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- pickup_eta_minutes: double (nullable = true)
 |-- pickup_time_delay: double (nullable = true)
 |-- pickup_distance_km: double (nullable = t

### Anomaly Detection Features

#### Identify delays based on time threshold deviations

In [82]:
# Step 1: Define the threshold for delay
time_threshold = 360  # Set the threshold in minutes

# Step 2: Identify delays based on the pickup time delay
# We create a new column `is_delayed` that indicates if the delay is above the threshold
predictions = predictions.withColumn(
    "is_delayed", 
    F.when(F.col("pickup_time_delay") > time_threshold, "Delayed").otherwise("On Time")
)

# Show the resulting DataFrame with the delay identification
predictions.select(
    "order_id", 
    "city", 
    "pickup_time_delay", 
    "pickup_time", 
    "accept_time", 
    "is_delayed"
).show()

+--------+---------+-----------------+-------------------+-------------------+----------+
|order_id|     city|pickup_time_delay|        pickup_time|        accept_time|is_delayed|
+--------+---------+-----------------+-------------------+-------------------+----------+
|  998888|    Jilin|            176.0|2025-10-22 12:08:00|2025-10-22 09:12:00|   On Time|
| 1817267|Chongqing|             27.0|2025-10-22 09:39:00|2025-10-22 09:12:00|   On Time|
| 5524389|   Yantai|             27.0|2025-10-22 09:39:00|2025-10-22 09:12:00|   On Time|
| 3124312|Chongqing|            132.0|2025-10-22 11:24:00|2025-10-22 09:12:00|   On Time|
|  221410| Shanghai|            106.0|2025-10-22 10:58:00|2025-10-22 09:12:00|   On Time|
| 3287344|Chongqing|            115.0|2025-10-22 11:07:00|2025-10-22 09:12:00|   On Time|
|  964270| Shanghai|            389.0|2025-10-22 15:41:00|2025-10-22 09:12:00|   Delayed|
| 3835946|   Yantai|            176.0|2025-10-22 12:08:00|2025-10-22 09:12:00|   On Time|
| 3408841|

#### Flag inconsistent geospatial entries (e.g., unrealistic speed between locations).

In [83]:
# Step 1: Convert pickup_time_delay from minutes to hours
predictions = predictions.withColumn(
    "time_diff_hours", 
    F.col("pickup_time_delay") / 60  # Convert minutes to hours
)

# Step 2: Calculate the speed in km/h using the existing pickup_distance_km column
predictions = predictions.withColumn(
    "speed_kmh", 
    F.when(F.col("time_diff_hours") > 0, F.floor(F.col("pickup_distance_km") / F.col("time_diff_hours"))).otherwise(0)
)

# Step 3: Flag entries with unrealistic speed
speed_threshold = 100  # Define a reasonable speed threshold (in km/h)
predictions = predictions.withColumn(
    "speed_status", 
    F.when(F.col("speed_kmh") > speed_threshold, "Unrealistic Speed").otherwise("Realistic Speed")
)

predictions = predictions.drop("time_diff_hours")

# Show the resulting DataFrame with the speed status column
predictions.select(
    "order_id", 
    "city", 
    "pickup_distance_km", 
    "pickup_time_delay",  # Using existing column for time delay in minutes
    "speed_kmh", 
    "speed_status"
).show()

+--------+---------+------------------+-----------------+---------+-----------------+
|order_id|     city|pickup_distance_km|pickup_time_delay|speed_kmh|     speed_status|
+--------+---------+------------------+-----------------+---------+-----------------+
|  998888|    Jilin|              0.19|            176.0|        0|  Realistic Speed|
| 1817267|Chongqing|              0.34|             27.0|        0|  Realistic Speed|
| 5524389|   Yantai|            510.99|             27.0|     1135|Unrealistic Speed|
| 3124312|Chongqing|             20.53|            132.0|        9|  Realistic Speed|
|  221410| Shanghai|              0.64|            106.0|        0|  Realistic Speed|
| 3287344|Chongqing|              0.49|            115.0|        0|  Realistic Speed|
|  964270| Shanghai|              0.45|            389.0|        0|  Realistic Speed|
| 3835946|   Yantai|              0.56|            176.0|        0|  Realistic Speed|
| 3408841|   Yantai|            590.97|           1833

### Save Data

In [84]:
predictions = predictions.drop('features','scaled_features')

In [85]:
# from pyspark.sql.functions import udf

# # Convert vector column to array of doubles (e.g., 'features' or 'scaled_features')
# def vector_to_array(vec):
#     if isinstance(vec, Vector):
#         return vec.toArray().tolist()
#     else:
#         return []

# # Register the UDF
# vector_to_array_udf = udf(vector_to_array, ArrayType(DoubleType()))

# # Apply the UDF to your vector columns and convert them into array of doubles
# predictions = predictions.withColumn("pca_features_array", vector_to_array_udf("pca_features"))

# # Drop the original vector columns (if you no longer need them)
# predictions = predictions.drop("pca_features")  # Keep 'pca_features_array'

In [86]:
print(predictions.columns) 

['city', 'hour_of_day', 'courier_id', 'order_id', 'region_id', 'accept_time', 'time_window_start', 'time_window_end', 'aoi_id', 'aoi_type', 'pickup_time', 'ds', 'lng', 'lat', 'pickup_gps_lng', 'pickup_gps_lat', 'accept_gps_lng', 'accept_gps_lat', 'day_of_week', 'month', 'pickup_eta_minutes', 'pickup_time_delay', 'pickup_distance_km', 'cluster', 'avg_pickup_time_minutes', 'pickup_order_count', 'city_order_count', 'is_delayed', 'speed_kmh', 'speed_status']


In [87]:
selected_columns = [
    "order_id",
    "pickup_time",
    "accept_time",
    "hour_of_day",
    "day_of_week",
    "month",
    "pickup_eta_minutes",
    "pickup_time_delay",
    "pickup_distance_km",
    "cluster",
    "avg_pickup_time_minutes",
    "pickup_order_count",
    "city",
    "city_order_count",
    "is_delayed",
    "speed_kmh",
    "speed_status",
]

# Create a new DataFrame with only the selected columns
final_predictions = predictions.select(*selected_columns)

# Define PostgreSQL connection details
jdbc_url = "jdbc:postgresql://localhost:5432/postgres"
properties = {
    "user": "postgres",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

final_predictions.write.jdbc(
    url=jdbc_url,
    table="feature_engg_pickup_data",
    mode="overwrite",
    properties=properties,
)
print("Data Saved to Postgresql successfully")

Data Saved to Postgresql successfully


In [88]:
# Save the cleaned DataFrame to csv
output_path = r'C:\Users\Dusty\Downloads\Internship\Last-Mile-Delivery-Delays-and-Route-Optimization\data\feature_engg_pickup_data.csv'
final_predictions.coalesce(1).write.option("header", "true").csv(output_path, mode="overwrite")
print("Data saved to csv successfully")

Data saved to csv successfully
